<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [177]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns

In [153]:
df = pd.read_csv('house-votes-84.data',header=None)

In [154]:
df.replace('?',np.nan,inplace=True)

In [155]:
cols = ['dem/rep','handicapped_infants','water-project-cost-sharing','adoption-of-the-budget-resolution',
'physician-fee-freeze','el-salvador-aid','religious-groups-in-schools','anti-satellite-test-ban',
'aid-to-nicaraguan-contras','mx-missile','immigration','synfuels-corporation-cutback','education-spending',
'superfund-right-to-sue','crime','duty-free-exports','export-administration-act-south-africa']

In [156]:
df.columns = cols

In [157]:
df.isnull().sum()

dem/rep                                     0
handicapped_infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64

In [158]:
df.head()

,dem/rep,handicapped_infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


## Filling NaN values with mode depending on dem/rep

In [159]:
# Let's try imputing missing data instead of deleting. We'll assume NaNs will vote with their party's majority

In [160]:
rep = df[df['dem/rep'] == 'republican']
dem = df[df['dem/rep'] == 'democrat']

In [161]:
def fill_with_mode(df):
    for col in df.columns:
        df[col].fillna(df[col].mode()[0], inplace=True)
    return df

In [162]:
rep = rep.replace('y',1).replace('n',0)

In [163]:
dem = dem.replace('y',1).replace('n',0)

In [164]:
rep = fill_with_mode(rep)

In [165]:
dem = fill_with_mode(dem)

In [166]:
# merging back into one df

In [167]:
df = pd.concat([rep,dem]).sort_index()

## Running t-tests

### Dems supporting more than rep

In [168]:
dem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 2 to 431
Data columns (total 17 columns):
dem/rep                                   267 non-null object
handicapped_infants                       267 non-null float64
water-project-cost-sharing                267 non-null float64
adoption-of-the-budget-resolution         267 non-null float64
physician-fee-freeze                      267 non-null float64
el-salvador-aid                           267 non-null float64
religious-groups-in-schools               267 non-null float64
anti-satellite-test-ban                   267 non-null float64
aid-to-nicaraguan-contras                 267 non-null float64
mx-missile                                267 non-null float64
immigration                               267 non-null float64
synfuels-corporation-cutback              267 non-null float64
education-spending                        267 non-null float64
superfund-right-to-sue                    267 non-null float64
crime         

In [169]:
rep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 0 to 434
Data columns (total 17 columns):
dem/rep                                   168 non-null object
handicapped_infants                       168 non-null float64
water-project-cost-sharing                168 non-null float64
adoption-of-the-budget-resolution         168 non-null float64
physician-fee-freeze                      168 non-null float64
el-salvador-aid                           168 non-null float64
religious-groups-in-schools               168 non-null float64
anti-satellite-test-ban                   168 non-null float64
aid-to-nicaraguan-contras                 168 non-null float64
mx-missile                                168 non-null float64
immigration                               168 non-null float64
synfuels-corporation-cutback              168 non-null float64
education-spending                        168 non-null float64
superfund-right-to-sue                    168 non-null float64
crime         

In [170]:
df.describe()

,handicapped_infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000
mean,0.450575,0.558621,0.597701,0.413793,0.494253,0.629885,0.567816,0.565517,0.519540,0.503448,0.372414,0.422989,0.503448,0.586207,0.436782,0.857471
std,0.498124,0.497123,0.490926,0.493079,0.500543,0.483391,0.495950,0.496260,0.500193,0.500564,0.484004,0.494602,0.500564,0.493079,0.496558,0.349994
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [171]:
def create_t_test_set(null_df, reject_df):
    d1 = {}
    d1 = d1.fromkeys(null_df.columns)
    for col in df.columns[1:]:
        stat, p = stats.ttest_ind(null_df[col], reject_df[col])
        d1[col] = [stat,p]
    test_df = pd.DataFrame(d1, index=['t_test','p_value'])
    return test_df

In [174]:
stats.ttest_ind(rep['handicapped_infants'],dem['handicapped_infants'])

Ttest_indResult(statistic=-9.746279241312045, pvalue=2.0074960037996152e-20)

In [175]:
stats.ttest_ind(dem['handicapped_infants'],rep['handicapped_infants'])

Ttest_indResult(statistic=9.746279241312045, pvalue=2.0074960037996152e-20)

In [172]:
create_t_test_set(dem,rep)

,dem/rep,handicapped_infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
t_test,None,9.746279e+00,-0.227900,2.396145e+01,-5.064138e+01,-2.202219e+01,-1.024774e+01,1.326349e+01,1.910790e+01,1.757738e+01,-2.057707,9.243172e+00,-2.257689e+01,-1.481644e+01,-1.722634e+01,1.425913e+01,7.823009e+00
p_value,None,2.007496e-20,0.819832,2.159693e-81,4.976028e-184,1.168664e-72,3.285781e-22,6.208204e-34,1.798550e-59,1.361418e-52,0.040216,1.091794e-18,3.649939e-75,1.807622e-40,5.014839e-51,4.290873e-38,3.981183e-14


In [173]:
create_t_test_set(rep,dem)

,dem/rep,handicapped_infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
t_test,None,-9.746279e+00,0.227900,-2.396145e+01,5.064138e+01,2.202219e+01,1.024774e+01,-1.326349e+01,-1.910790e+01,-1.757738e+01,2.057707,-9.243172e+00,2.257689e+01,1.481644e+01,1.722634e+01,-1.425913e+01,-7.823009e+00
p_value,None,2.007496e-20,0.819832,2.159693e-81,4.976028e-184,1.168664e-72,3.285781e-22,6.208204e-34,1.798550e-59,1.361418e-52,0.040216,1.091794e-18,3.649939e-75,1.807622e-40,5.014839e-51,4.290873e-38,3.981183e-14


In [196]:
dem['religious-groups-in-schools'].value_counts()

0.0    144
1.0    123
Name: religious-groups-in-schools, dtype: int64

In [197]:
rep['religious-groups-in-schools'].value_counts()

1.0    151
0.0     17
Name: religious-groups-in-schools, dtype: int64

### Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

religious-groups-in-schools

### Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

Handicapped infants

### Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference

water-project-cost-sharing